In [1]:
!pip install ultralytics
!pip install ultralytics Pillow

In [2]:
!pip install ultralytics
!pip install opencv-python
!pip install pillow
!pip install tqdm
!pip install pyyaml


In [ ]:
import base64
import io
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms, models
from fastapi import FastAPI
from pydantic import BaseModel
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],   # 또는 ['http://localhost:4478']
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


# ================================
# 1) YOLO 얼굴 검출 모델 로드
# ================================
print("📥 YOLOv8 Face 모델 다운로드 중...")

yolo_path = hf_hub_download(
    repo_id="Reshma67/yolov8-face-detection",
    filename="model.pt"
)
yolo_model = YOLO(yolo_path)

# ================================
# 2) ResNetCBAM 정의
# ================================
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv1(x_cat))


class CBAM(nn.Module):
    def __init__(self, in_planes):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = x * self.ca(x)
        x = x * self.sa(x)
        return x


class ResNetCBAM(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetCBAM, self).__init__()

        base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

        self.conv1 = base.conv1
        self.bn1 = base.bn1
        self.relu = base.relu
        self.maxpool = base.maxpool

        self.layer1 = base.layer1
        self.layer2 = base.layer2
        self.cbam2 = CBAM(128)
        self.layer3 = base.layer3
        self.cbam3 = CBAM(256)
        self.layer4 = base.layer4

        self.avgpool = base.avgpool
        self.fc = nn.Linear(base.fc.in_features, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.cbam2(x)
        x = self.layer3(x)
        x = self.cbam3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


# ================================
# 3) ResNetCBAM 모델 로드
# ================================
emotion_model = ResNetCBAM(num_classes=2)
emotion_model.load_state_dict(torch.load("Res_CBAM_v3_epc50.pth", map_location="cuda"))
emotion_model.eval()
emotion_model.cuda()

# ================================
# 4) 이미지 변환
# ================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

# ================================
# 5) 요청 body
# ================================
class Frame(BaseModel):
    image: str  # base64


@app.post("/predict")
def predict(frame: Frame):
    base64_data = frame.image.split(",")[-1]
    img_bytes = base64.b64decode(base64_data)
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
    img_np = np.array(img)

    # YOLO 얼굴 검출
    results = yolo_model.predict(img_np, imgsz=320, device=0, verbose=False)
    boxes = results[0].boxes.xyxy.cpu().numpy()

    if len(boxes) == 0:
        print("⛔ No face detected → OTHER(0.0)")
        return {"emotion": "other", "prob": 0.0}

    x1, y1, x2, y2 = sorted(
        boxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True
    )[0]
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

    crop = img_np[y1:y2, x1:x2]
    if crop.size == 0:
        print("⛔ Crop empty → OTHER(0.0)")
        return {"emotion": "other", "prob": 0.0}

    # Transform + Predict
    crop_tensor = transform(Image.fromarray(crop)).unsqueeze(0).cuda()

    with torch.no_grad():
        output = emotion_model(crop_tensor)
        prob = torch.softmax(output, dim=1)[0][1].item()

        label = "laugh" if prob > 0.5 else "other"
        print(f"PREDICT → {label.upper()} ({prob:.4f})")

        return {"emotion": label, "prob": prob}


전체코드

In [ ]:
import base64
import io
import cv2
import numpy as np
from PIL import Image
import torch

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware

# Ultralytics (YOLO) & HuggingFace
from ultralytics import YOLO
from huggingface_hub import hf_hub_download

# ================================
# 0) 설정 및 앱 초기화
# ================================
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 보안상 필요시 특정 도메인으로 제한 권장
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 디바이스 설정 (GPU 우선)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🚀 Device set to: {device}")

# ================================
# 1) YOLO 얼굴 검출 모델 로드
# ================================
print("📥 [1/2] Loading Face Detection Model (YOLOv8-Face)...")

try:
    # HuggingFace에서 모델 다운로드 (최초 1회만 다운로드됨)
    face_model_path = hf_hub_download(
        repo_id="Reshma67/yolov8-face-detection",
        filename="model.pt"
    )
    face_model = YOLO(face_model_path)
    print("✅ Face Model Loaded.")
except Exception as e:
    print(f"❌ Face Model Load Error: {e}")
    raise e


# ================================
# 2) YOLO 감정 분류 모델 (Da4) 로드
# ================================
# [중요] 학습된 모델 파일명이 'Da4.pt' 혹은 'best.pt'라고 가정합니다.
# 실제 파일 경로에 맞게 수정해주세요.
EMOTION_MODEL_PATH = "Da4.pt" 

print(f"📥 [2/2] Loading Emotion Classification Model ({EMOTION_MODEL_PATH})...")

try:
    # 사용자 지정 YOLO 분류 모델 로드
    emotion_model = YOLO(EMOTION_MODEL_PATH)
    emotion_model.to(device)
    print("✅ Emotion Model Loaded.")
except Exception as e:
    print(f"❌ Emotion Model Load Error (경로를 확인하세요): {e}")
    # 파일이 없을 경우를 대비해 예외 처리 필요 (테스트용 더미 로직 등)
    emotion_model = None

# ================================
# 3) 요청 데이터 모델 정의
# ================================
class Frame(BaseModel):
    image: str  # Base64 Encoded Image String


# ================================
# 4) 추론 엔드포인트
# ================================
@app.post("/predict")
def predict(frame: Frame):
    """
    이미지를 받아 얼굴을 검출하고, 해당 얼굴의 감정을 분류하여 반환합니다.
    """
    # 1. Base64 디코딩 -> 이미지 변환
    try:
        base64_data = frame.image.split(",")[-1]
        img_bytes = base64.b64decode(base64_data)
        pil_img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        img_np = np.array(pil_img)
    except Exception as e:
        return {"emotion": "error", "prob": 0.0, "msg": "Invalid Image Data"}

    # 2. 얼굴 검출 (Face Detection)
    # imgsz=320 등으로 속도 최적화 가능
    face_results = face_model.predict(img_np, imgsz=320, device=device, verbose=False, conf=0.5)
    boxes = face_results[0].boxes.xyxy.cpu().numpy()

    # 얼굴이 없으면 'other' 반환
    if len(boxes) == 0:
        print("⛔ No face detected")
        return {"emotion": "other", "prob": 0.0}

    # 3. 가장 큰 얼굴 영역 선택 (중심 혹은 면적 기준)
    # (x1, y1, x2, y2)
    largest_face = sorted(boxes, key=lambda b: (b[2] - b[0]) * (b[3] - b[1]), reverse=True)[0]
    x1, y1, x2, y2 = map(int, largest_face)

    # 좌표가 이미지 범위를 벗어나지 않도록 클립
    h, w, _ = img_np.shape
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)

    # 4. 얼굴 영역 Crop
    face_crop = img_np[y1:y2, x1:x2]
    
    # Crop 된 이미지가 너무 작거나 비어있으면 처리
    if face_crop.size == 0 or face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
        print("⛔ Face crop too small")
        return {"emotion": "other", "prob": 0.0}

    # 5. 감정 분류 (Emotion Classification - Da4 Model)
    if emotion_model:
        # YOLO 분류 모델은 내부적으로 리사이징 및 정규화를 수행하므로 원본 Crop 전달 가능
        cls_results = emotion_model.predict(face_crop, device=device, verbose=False)
        
        # 결과 파싱
        probs = cls_results[0].probs  # 확률 객체
        top1_index = probs.top1       # 가장 높은 확률의 클래스 인덱스
        top1_conf = probs.top1conf.item() # 가장 높은 확률 값 (tensor -> float)
        
        # 클래스 이름 매핑 (모델 내부에 저장된 names 사용)
        class_names = cls_results[0].names
        pred_label = class_names[top1_index]

        # 확률값 포맷팅
        prob_val = round(top1_conf, 4)

        print(f"👁️ Detect: {pred_label.upper()} ({prob_val})")

        return {
            "emotion": pred_label,  # 예: 'happy', 'anger', 'neutral' 등
            "prob": prob_val
        }
    else:
        # 모델 로드 실패 시
        return {"emotion": "error", "prob": 0.0, "msg": "Model not loaded"}



crop+yolo모델

In [ ]:
import os
import json
import csv
import random
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import f1_score

# ==============================================================
# 1. 설정 및 데이터 경로 (요청하신 주소로 업데이트됨)
# ==============================================================
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS = 50
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
IMG_SIZE = 224
MODEL_SAVE_PATH = "best_emotion_model.pt"
CSV_PATH = "training_log.csv"
IMG_SAVE_DIR = Path("epoch_previews")
IMG_SAVE_DIR.mkdir(exist_ok=True)

# 클래스 정의
CLASSES = ["happy", "average", "other"]
CLASS_TO_ID = {cls: i for i, cls in enumerate(CLASSES)}
ID_TO_CLASS = {i: cls for i, cls in enumerate(CLASSES)}

# [요청] 업데이트된 학습 데이터 경로
TRAIN_SOURCES = {
    "happy": {
        "happy": {
            "img_dir": "/workspace/user4/segmented_output_bisenet/train/happy",
            "json_path": "/workspace/merge_data/new_data/label/happy_half.json"
        }
    },
    "average": {
        "anger": {
            "img_dir": "/workspace/user4/segmented_output_bisenet/train/anger",
            "json_path": "/workspace/merge_data/data/label/train/train_anger.json"
        }
    },
    "other": {
        "panic": {
            "img_dir": "/workspace/user4/segmented_output_bisenet/train/panic",
            "json_path": "/workspace/merge_data/data/label/train/train_panic.json"
        },
        "sadness": {
            "img_dir": "/workspace/user4/segmented_output_bisenet/train/sadness",
            "json_path": "/workspace/merge_data/data/label/train/train_sadness.json"
        },
    }
}

# [요청] 업데이트된 검증 데이터 경로
human = {
    "happy": {
        "happy": {
            "img_dir": "/workspace/user4/segmented_output_bisenet/val/happy",
           
        }
    }
   
}

# ==============================================================
# 2. 데이터셋 클래스 및 유틸리티
# ==============================================================
class EmotionDataset(Dataset):
    def __init__(self, sources, transform=None):
        self.transform = transform
        self.samples = []  # (img_path, label_id, confidence)

        for label_str, sub_dict in sources.items():
            label_id = CLASS_TO_ID[label_str]
            
            for sub_emotion, paths in sub_dict.items():
                img_dir = Path(paths["img_dir"])
                json_path = paths["json_path"]
                
                # JSON 로드 (파일 구조에 따라 수정 필요할 수 있음)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                        
                    # JSON 구조가 리스트인지 딕셔너리인지에 따라 처리
                    # 가정: [{"filename": "a.jpg", "confidence": 1.0}, ...] 형태라고 가정
                    # 실제 데이터 포맷에 맞게 파싱 로직을 조정하세요.
                    items = data if isinstance(data, list) else data.get('images', [])
                    
                    for item in items:
                        # 파일명 키 찾기 (filename, file_name 등)
                        fname = item.get('filename') or item.get('file_name')
                        if not fname: continue
                        
                        # confidence 키 (없으면 1.0)
                        conf = float(item.get('confidence', 1.0))
                        
                        full_path = img_dir / fname
                        if full_path.exists():
                            self.samples.append((str(full_path), label_id, conf))
                            
                except Exception as e:
                    print(f"⚠️ Warning: Failed to load {json_path} or parse data: {e}")

        print(f"✅ Loaded {len(self.samples)} samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_id, conf = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except Exception:
            # 에러 발생 시 검은색 이미지 반환
            image = torch.zeros((3, IMG_SIZE, IMG_SIZE))
        
        return image, torch.tensor(label_id, dtype=torch.long), torch.tensor(conf, dtype=torch.float)


def get_transforms(is_train=True):
    if is_train:
        return transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            # ImageNet Norm
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# ==============================================================
# 3. [요청 기능] 시각화 함수 (파란색 박스 스타일)
# ==============================================================
def save_epoch_preview(model, val_loader, epoch, device, save_dir):
    """
    검증 배치를 하나 가져와서 시각화 이미지를 저장합니다.
    요청하신 파란색 배경 텍스트 스타일을 적용했습니다.
    """
    model.eval()
    try:
        # 배치 하나 가져오기
        imgs, lbls, confs = next(iter(val_loader))
    except StopIteration:
        return

    imgs, lbls = imgs.to(device), lbls.to(device)
    
    with torch.no_grad():
        logits = model(imgs)
        probs = F.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)
        conf_values, _ = torch.max(probs, dim=1) # 모델이 예측한 확률

    # 역정규화 (시각화를 위해)
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(device)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(device)
    imgs_vis = imgs * std + mean
    imgs_vis = torch.clamp(imgs_vis, 0, 1)

    # [요청하신 Matplotlib 코드 적용]
    cols = 4
    rows = 4
    plt.figure(figsize=(12, 12))
    
    batch_limit = min(cols * rows, len(imgs))
    
    for i in range(batch_limit):
        ax = plt.subplot(rows, cols, i+1)
        # Tensor -> Numpy (H, W, C)
        img_np = imgs_vis[i].cpu().permute(1, 2, 0).numpy()
        plt.imshow(img_np)
        
        pred_cls = ID_TO_CLASS[preds[i].item()]
        true_cls = ID_TO_CLASS[lbls[i].item()]
        model_conf = conf_values[i].item()
        
        # 텍스트: P(예측), T(정답), 확률
        label_text = f"P:{pred_cls}\nT:{true_cls}\n{model_conf:.2f}"
        
        # [스타일] 파란색 배경, 흰색 글씨, 좌측 상단 (요청사항)
        plt.text(
            x=5, y=5,
            s=label_text,
            color='white',
            fontsize=9,
            fontweight='bold',
            verticalalignment='top',
            bbox=dict(facecolor='blue', alpha=0.8, edgecolor='none', pad=2)
        )
        plt.axis('off')
    
    plt.tight_layout()
    save_path = save_dir / f"epoch_{epoch:03d}_preview.png"
    plt.savefig(save_path)
    plt.close()
    print(f" 🖼️ Preview saved: {save_path}")


# ==============================================================
# 4. 메인 학습 루프
# ==============================================================
def main():
    print(f"🚀 Device: {DEVICE}")
    
    # 데이터셋 로드
    print("Loading Train Data...")
    train_dataset = EmotionDataset(TRAIN_SOURCES, transform=get_transforms(is_train=True))
    print("Loading Val Data...")
    val_dataset = EmotionDataset(human, transform=get_transforms(is_train=False))
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    # 모델 정의 (ResNet18 예시)
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(model.fc.in_features, len(CLASSES))
    model.to(DEVICE)

    criterion = nn.CrossEntropyLoss(reduction='none') # Weak supervision을 위해 none 설정
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    # CSV 헤더 작성
    with open(CSV_PATH, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Train_Loss', 'Val_Loss', 'Val_Acc', 'F1_Macro', 'F1_Happy', 'F1_Avg', 'F1_Other'])

    # [요청] best_f1 초기화
    best_f1 = 0.0

    print("🏁 Start Training...")

    for epoch in range(NUM_EPOCHS):
        # --- Train ---
        model.train()
        train_loss = 0.0
        
        loop = tqdm(train_loader, desc=f"Ep {epoch+1}/{NUM_EPOCHS}")
        for imgs, lbls, confs in loop:
            imgs, lbls, confs = imgs.to(DEVICE), lbls.to(DEVICE), confs.to(DEVICE)
            
            optimizer.zero_grad()
            logits = model(imgs)
            
            # Loss 계산
            raw_loss = criterion(logits, lbls)
            # [요청] Weak Supervision: Loss * Confidence
            loss = (raw_loss * confs).mean()
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
        
        scheduler.step()
        
        # --- Validation ---
        model.eval()
        val_loss = 0.0
        preds_list, trues_list = [], []
        
        with torch.no_grad():
            for imgs, lbls, confs in val_loader:
                imgs, lbls, confs = imgs.to(DEVICE), lbls.to(DEVICE), confs.to(DEVICE)
                
                logits = model(imgs)
                raw_loss = criterion(logits, lbls)
                loss = (raw_loss * confs).mean()
                val_loss += loss.item()
                
                probs = F.softmax(logits, dim=1)
                preds = torch.argmax(probs, dim=1)
                
                preds_list.extend(preds.cpu().numpy())
                trues_list.extend(lbls.cpu().numpy())

        # --- Metrics ---
        val_acc = (np.array(preds_list) == np.array(trues_list)).mean()
        f1_macro = f1_score(trues_list, preds_list, average='macro', zero_division=0)
        f1_per_class = f1_score(trues_list, preds_list, average=None, zero_division=0)
        
        # 클래스별 F1 Score (인덱스 순서에 주의)
        happy_f1 = f1_per_class[CLASS_TO_ID["happy"]] if len(f1_per_class) > CLASS_TO_ID["happy"] else 0.0
        avg_f1 = f1_per_class[CLASS_TO_ID["average"]] if len(f1_per_class) > CLASS_TO_ID["average"] else 0.0
        other_f1 = f1_per_class[CLASS_TO_ID["other"]] if len(f1_per_class) > CLASS_TO_ID["other"] else 0.0

        print(f" -> Val Acc: {val_acc:.4f} | F1 Macro: {f1_macro:.4f}")
        print(f" -> [Happy]: {happy_f1:.4f} | [Avg]: {avg_f1:.4f} | [Other]: {other_f1:.4f}")

        # --- Logging ---
        with open(CSV_PATH, 'a', newline='') as f:
            csv.writer(f).writerow([
                epoch+1, 
                train_loss/len(train_loader), 
                val_loss/len(val_loader), 
                val_acc, 
                f1_macro, 
                happy_f1, 
                avg_f1, 
                other_f1
            ])
        
        # [요청] 시각화 저장 (파란 박스 스타일 적용됨)
        save_epoch_preview(model, val_loader, epoch+1, DEVICE, IMG_SAVE_DIR)

        # [요청] 모델 저장 (Best F1 기준)
        if f1_macro > best_f1:
            best_f1 = f1_macro
            torch.save(model.state_dict(), MODEL_SAVE_PATH)
            print(f" -> ⭐ Best Model Saved! (New F1: {best_f1:.4f})")

if __name__ == '__main__':
    main()